In [1]:
load '../../../global/datasets/WallFollowingRobot.py'

In [2]:
load '../../../global/samplers/RandomTrainTestSampler.py'

In [3]:
import numpy as np
import torch

class IncreaseDimension(object):

    def __call__(self, o):

        return np.expand_dims(o, axis=0)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv1d(1, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 11, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 11)
        x = self.classifier(x)
        return x

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets
import torchvision.models as models

transformsData = transforms.Compose([IncreaseDimension()] )

dataset = WallFollowingRobot(root='./data', transform=transformsData, download=True)

sampler = RandomTrainTestSampler(dataset)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=sampler.trainSampler(), num_workers=2 )
testloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=sampler.testSampler(), num_workers=2)

net = Net()
net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print("Start training")
for epoch in range(300):

    running_loss = 0.0
    for data in trainloader:

        # get the inputs
        inputs, labels = data
        
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
       
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    print(loss.data[0])

    correct = 0
    total = 0
    
    for data in testloader:
  
        inputs, labels = data
        
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        outputs = net(Variable(inputs))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy of the network on the test images: %f %%' % (100 * correct / total))

print('Finished Training')




Files already downloaded and verified
Start training
0.9305362105369568
Accuracy of the network on the test images: 53.113553 %
0.848880410194397
Accuracy of the network on the test images: 54.578755 %
0.8078418374061584
Accuracy of the network on the test images: 56.868132 %
0.9830004572868347
Accuracy of the network on the test images: 59.798535 %
0.9468162655830383
Accuracy of the network on the test images: 65.384615 %
0.9543581008911133
Accuracy of the network on the test images: 71.336996 %
0.8055624961853027
Accuracy of the network on the test images: 74.358974 %
0.437275230884552
Accuracy of the network on the test images: 75.824176 %
0.3488006591796875
Accuracy of the network on the test images: 77.197802 %
0.3036573529243469
Accuracy of the network on the test images: 78.663004 %
0.2878172695636749
Accuracy of the network on the test images: 80.402930 %
0.17109329998493195
Accuracy of the network on the test images: 84.890110 %
0.07503367960453033
Accuracy of the network on t